Parse CAGI5 data to get WT and mutant sequences for zero-shot prediction with ResidualBind models.

Includes code to identify and add in missing alternate alleles.

In [1]:
import numpy as np
import os, h5py
import pandas as pd
from collections import defaultdict
import glob
import subprocess

# Helper functions

In [2]:
def enforce_const_range(site, window):
    """
    Function to get constant size bed ranges
    :param site: center positions
    :param window: window size around center position
    :return: new starts and ends
    """
    half_window = window//2
    start = site - half_window
    end = site + half_window
    return start, end

def expand_range(bedfile, output_filename, window=3072):
    """
    Function to write a new bed file with expanded ranges
    :param bedfile: existing bed file, the ranges of which will be expanded
    :param output_filename: new bed file path
    :param window: window size
    :return: None
    """
    df = pd.read_table(bedfile, header=None, index_col=None)
    start, end = enforce_const_range(df.iloc[:,1].astype(int), window)
    df_expanded = df.copy()
    df_expanded.iloc[:,1] = start.values
    df_expanded.iloc[:,2] = end.values
    df_nonneg = df_expanded[df_expanded.iloc[:,1]>0]
    df_nonneg = df_nonneg.reset_index(drop=True)
    # add string identifier (chr:start-end/ref>alt)
    df_nonneg['identifier'] = [f'{df_nonneg.iloc[i,0]}:{df_nonneg.iloc[i,1]}-{df_nonneg.iloc[i,2]}/{df_nonneg.iloc[i,3]}>{df_nonneg.iloc[i,4]}' for i in range(df_nonneg.shape[0])] 
    df_nonneg.to_csv(output_filename, header=None, sep='\t', index=None)

def convert_bed_to_seq(bedfile, output_fa, genomefile):
    """
    This function collects DNA sequences corresponding to a bedfile into a fasta file
    :param bedfile: existing bed file
    :param output_fa: new fasta path (generated if it doesn't exist)
    :param genomefile: genome fasta to use to get the sequences
    :return: list of coordinates and string sequences
    """
    if os.path.isfile(output_fa) is not True:
        '''
        generate fasta file if it doesn't exist yet 
        '''
        cmd = 'bedtools getfasta -fi {} -bed {} -s -fo {}'.format(genomefile, bedfile, output_fa)
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
        _ = process.communicate()
    coords_list, seqs_list = fasta2list(output_fa)
    return coords_list, seqs_list


def fasta2list(fasta_file):
    """
    Function to convert fasta file to a list of DNA strings
    :param fasta_file: existing fasta file
    :return: list of coordinates and string sequences
    """
    fasta_coords = []
    seqs = []
    # header = ''

    for line in open(fasta_file):
        if line[0] == '>':
            # header = line.split()[0][1:]
            fasta_coords.append(line[1:].rstrip())
        else:
            s = line.rstrip()
            s = s.upper()
            seqs.append(s)

    return fasta_coords, seqs

def dna_one_hot(seq):
    """
    Function to convert string DNA sequences into onehot
    :param seq: string DNA sequence
    :return: onehot sequence
    """
    seq_len = len(seq)
    seq_start = 0
    seq = seq.upper()

    # map nt's to a matrix len(seq)x4 of 0's and 1's.
    seq_code = np.zeros((seq_len, 4), dtype='float16')

    for i in range(seq_len):
        if i >= seq_start and i - seq_start < len(seq):
            nt = seq[i - seq_start]
            if nt == 'A':
                seq_code[i, 0] = 1
            elif nt == 'C':
                seq_code[i, 1] = 1
            elif nt == 'G':
                seq_code[i, 2] = 1
            elif nt == 'T':
                seq_code[i, 3] = 1
            else:
                seq_code[i, :] = 0.25

    return seq_code

# Parse combined BED files from CAGI5 challenge + release data

In [3]:
'''
for each enhancer/promoter (regulator)
combine challenge and release data as a single bed file 
'''

all_dfs = defaultdict(list)
cagi_data = '../data/cagi5/'
combined_filename = '../data/cagi5/combined_cagi.bed'
for filename in glob.glob('../data/cagi5/*tsv'):
    prefix, regulator = os.path.basename(filename).split('.tsv')[0].split('_')
    one_reg = pd.read_table(filename, skiprows=7, header=None)
    one_reg['regulator'] = regulator
    one_reg['set'] = prefix
    all_dfs[regulator].append(one_reg)
    
for regulator, dfs in all_dfs.items():
    print(f'regulator: {regulator}')
    combined_filename = f'../data/cagi5/{regulator}_combined_cagi.bed'
    combined_cagi = pd.concat(dfs)
    combined_cagi.insert(4, 'strand', '+')
    combined_cagi.insert(2,'end',combined_cagi.iloc[:,1]+1)
    combined_cagi.iloc[:,0] = 'chr'+combined_cagi.iloc[:,0].astype(str)
    combined_cagi.iloc[:,1] = combined_cagi.iloc[:,1].astype(int)
    combined_cagi.iloc[:,2] = combined_cagi.iloc[:,1].astype(int)
    print(f'writing combined bed file to {combined_filename}')
    combined_cagi.to_csv(combined_filename, sep='\t', header=False, index=None)


regulator: IRF6
writing combined bed file to ../data/cagi5/IRF6_combined_cagi.bed
regulator: MYCrs6983267
writing combined bed file to ../data/cagi5/MYCrs6983267_combined_cagi.bed
regulator: F9
writing combined bed file to ../data/cagi5/F9_combined_cagi.bed
regulator: PKLR
writing combined bed file to ../data/cagi5/PKLR_combined_cagi.bed
regulator: GP1BB
writing combined bed file to ../data/cagi5/GP1BB_combined_cagi.bed
regulator: HNF4A
writing combined bed file to ../data/cagi5/HNF4A_combined_cagi.bed
regulator: IRF4
writing combined bed file to ../data/cagi5/IRF4_combined_cagi.bed
regulator: ZFAND3
writing combined bed file to ../data/cagi5/ZFAND3_combined_cagi.bed
regulator: TERT-HEK293T
writing combined bed file to ../data/cagi5/TERT-HEK293T_combined_cagi.bed
regulator: MSMB
writing combined bed file to ../data/cagi5/MSMB_combined_cagi.bed
regulator: HBG1
writing combined bed file to ../data/cagi5/HBG1_combined_cagi.bed
regulator: LDLR
writing combined bed file to ../data/cagi5/LDL

# Define seqlen for ResidualBind lentiMPRA models

In [4]:
# input length for ResidualBind lentiMPRA models
seqlen = 230 

# K562
## _PKLR_
### Generate BED file w/ expanded ranges



In [5]:
regulator = 'PKLR'

# output filename for expanded bed file
output_filename = f'../data/cagi5/{regulator}_combined_cagi_{seqlen}bp.bed'

if os.path.isfile(output_filename) is not True:
    # generate bed file with expanded range 
    expand_range(f'../data/cagi5/{regulator}_combined_cagi.bed', output_filename, window = seqlen)

### Get sequences and their coordinates

In [89]:
fa_filename = f'../data/cagi5/{regulator}_combined_cagi_{seqlen}bp.fa'
coords_list, seqs_list = convert_bed_to_seq(output_filename, fa_filename, genomefile='/shared/jessica/ref/hg19/hg19.fa')

### Generate one hot seqs (WT & Mutant)

In [90]:
bad_lines = []
nonneg_df = pd.read_csv(output_filename, sep='\t', header=None) # load bed file with ref/alt alleles 
onehot_ref = []
onehot_alt = []
identifiers = []
pos_dict = {'+': int(seqlen/2-1), '-':int(seqlen/2)} # ix of allele pos in seq
for i,(chr_s_e, seq) in enumerate(zip(coords_list, seqs_list)):
    alt = ''
    strand = chr_s_e.split('(')[-1].split(')')[0]
    pos = pos_dict[strand] # get allele pos 
    if seq[pos] != nonneg_df.iloc[i, 3]:
        # check that nucleotide at pos is ref allele 
        print('Error in line ' + str(i))
        bad_lines.append(i)
    else:
        # get alt allele 
        alt = nonneg_df.iloc[i,4]
        onehot = dna_one_hot(seq) # one hot wt seq 
        mutated_onehot = onehot.copy() 
        mutated_onehot[pos] = dna_one_hot(alt)[0] # one hot mutant seq 
        onehot_ref.append(onehot)
        onehot_alt.append(mutated_onehot) 
        identifiers.append(nonneg_df.iloc[i,-1])

onehot_alt = np.array(onehot_alt)
onehot_ref = np.array(onehot_ref)

In [92]:
# remove badlines 
included_df = nonneg_df[~nonneg_df.index.isin(bad_lines)]
included_df.to_csv(f'../data/cagi5/{regulator}_combined_cagi_{seqlen}bp_filt.bed', header=None, index=None, sep='\t')

### Write onehot seqs and identifiers to .h5 file


In [53]:
onehot_seqs_fh = f'../data/cagi5/{regulator}_onehot_{seqlen}bp.h5'
with h5py.File(onehot_seqs_fh, 'w') as fh:
    fh.create_dataset('ref', data=onehot_ref)
    fh.create_dataset('alt', data=onehot_alt)
    fh.create_dataset('identifier', data=identifiers)

### Examine h5 file 

In [74]:
with h5py.File(onehot_seqs_fh, 'r') as fh:
    ref_seqs = np.array(fh['ref'])
    alt_seqs = np.array(fh['alt'])
    seq_identifiers = list(fh['identifier'])


In [75]:
ref_seqs.shape

(1409, 230, 4)

In [76]:
alt_seqs.shape

(1409, 230, 4)

In [79]:
seq_identifiers = [x.decode('utf-8') for x in seq_identifiers]

### Add in missing alternate alleles

In [95]:
pklr_bed = pd.read_table(output_filename, header=None)
pklr_bed.head()

,0,1,2,3,4,5,6,7,8,9,10
0,chr1,155271119,155271349,A,C,+,0.07,0.00,PKLR,release,chr1:155271119-155271349/A>C
1,chr1,155271119,155271349,A,G,+,-0.67,0.85,PKLR,release,chr1:155271119-155271349/A>G
2,chr1,155271119,155271349,A,T,+,0.19,0.04,PKLR,release,chr1:155271119-155271349/A>T
3,chr1,155271120,155271350,G,A,+,-0.10,0.01,PKLR,release,chr1:155271120-155271350/G>A
4,chr1,155271120,155271350,G,C,+,0.06,0.00,PKLR,release,chr1:155271120-155271350/G>C


In [96]:
# identify missing alleles
result = pklr_bed.groupby([0,1,2,3])[4].nunique()
cases_with_missing_alleles = result[result!=3]
cases_with_missing_alleles

0     1          2          3
chr1  155271142  155271372  A    2
      155271251  155271481  C    2
      155271537  155271767  A    2
      155271540  155271770  T    2
Name: 4, dtype: int64

In [97]:
for i in range(len(cases_with_missing_alleles)):
    chrom, start, end, ref = cases_with_missing_alleles.index[i]
    sub_df = pklr_bed[(pklr_bed.iloc[:,0]==chrom) & (pklr_bed.iloc[:,1]==start)]
    alt = sub_df.iloc[:,4].tolist()
    alleles = set(alt + [ref])
    missing = (set(['A','C','G','T']) - alleles).pop()
    new_row = pd.DataFrame([chrom, start, end, ref, missing, '+', 'NA', 'NA', 'PKLR','missing', f'{chrom}:{start}-{end}/{ref}/{missing}']).transpose()
    print(new_row)
    loc = pklr_bed[(pklr_bed.iloc[:,0]==cases_with_missing_alleles.index[i][0]) & (pklr_bed.iloc[:,1]==cases_with_missing_alleles.index[i][1])].index[-1] + 1
    # insert new row
    print(f'inserting new row with missing alt allele {missing} at ix {loc}')
    pklr_bed = pd.concat([pklr_bed.iloc[:loc,:],
                          new_row,
                          pklr_bed.iloc[loc:,:]], ignore_index=True)

     0          1          2  3  4  5   6   7     8        9   \
0  chr1  155271142  155271372  A  C  +  NA  NA  PKLR  missing   

                             10  
0  chr1:155271142-155271372/A/C  
inserting new row with missing alt allele C at ix 551
     0          1          2  3  4  5   6   7     8        9   \
0  chr1  155271251  155271481  C  G  +  NA  NA  PKLR  missing   

                             10  
0  chr1:155271251-155271481/C/G  
inserting new row with missing alt allele G at ix 782
     0          1          2  3  4  5   6   7     8        9   \
0  chr1  155271537  155271767  A  C  +  NA  NA  PKLR  missing   

                             10  
0  chr1:155271537-155271767/A/C  
inserting new row with missing alt allele C at ix 1400
     0          1          2  3  4  5   6   7     8        9   \
0  chr1  155271540  155271770  T  G  +  NA  NA  PKLR  missing   

                             10  
0  chr1:155271540-155271770/T/G  
inserting new row with missing alt allele

In [106]:
# save to file 
pklr_bed.to_csv(f'../data/cagi5/{regulator}_combined_cagi_with_missing_alt_alleles_230bp.bed', header=None, sep='\t', index=None)

### Get sequences and their coordinates

In [107]:
fa_filename = f'../data/cagi5/{regulator}_cagi_with_missing_alt_alleles_{seqlen}bp.fa'
coords_list, seqs_list = convert_bed_to_seq('../data/cagi5/{regulator}_combined_cagi_with_missing_alt_alleles_230bp.bed', 
                                            fa_filename, genomefile='/shared/jessica/ref/hg19/hg19.fa')

### Generate one hot seqs (WT & Mutant)

In [109]:
bad_lines = []
nonneg_df = pd.read_csv(f'../data/cagi5/{regulator}_combined_cagi_with_missing_alt_alleles_230bp.bed', sep='\t', header=None) # load bed file with ref/alt alleles 
onehot_ref = []
onehot_alt = []
identifiers = []
pos_dict = {'+': int(seqlen/2-1), '-':int(seqlen/2)} # ix of allele pos in seq
for i,(chr_s_e, seq) in enumerate(zip(coords_list, seqs_list)):
    alt = ''
    strand = chr_s_e.split('(')[-1].split(')')[0]
    pos = pos_dict[strand] # get allele pos 
    if seq[pos] != nonneg_df.iloc[i, 3]:
        # check that nucleotide at pos is ref allele 
        print('Error in line ' + str(i))
        bad_lines.append(i)
    else:
        # get alt allele 
        alt = nonneg_df.iloc[i,4]
        onehot = dna_one_hot(seq) # one hot wt seq 
        mutated_onehot = onehot.copy() 
        mutated_onehot[pos] = dna_one_hot(alt)[0] # one hot mutant seq 
        onehot_ref.append(onehot)
        onehot_alt.append(mutated_onehot) 
        identifiers.append(nonneg_df.iloc[i,-1])

onehot_alt = np.array(onehot_alt)
onehot_ref = np.array(onehot_ref)

In [104]:
# remove badlines 
included_df = nonneg_df[~nonneg_df.index.isin(bad_lines)]
included_df.to_csv(f'../data/cagi5/{regulator}_combined_cagi_with_missing_alt_alleles_{seqlen}bp_filt.bed', header=None, index=None, sep='\t')

### Write onehot seqs and identifiers to .h5 file


In [110]:
onehot_seqs_fh = f'../data/cagi5/{regulator}_with_missing_alt_alleles_onehot_{seqlen}bp.h5'
with h5py.File(onehot_seqs_fh, 'w') as fh:
    fh.create_dataset('ref', data=onehot_ref)
    fh.create_dataset('alt', data=onehot_alt)
    fh.create_dataset('identifier', data=identifiers)

# HepG2
- _F9_
- _LDLR_
- _SORT1_
## _F9_

### Generate BED file w/ expanded ranges

In [93]:
regulator = 'F9'

# output filename for expanded bed file
output_filename = f'../data/cagi5/{regulator}_combined_cagi_{seqlen}bp.bed'

if os.path.isfile(output_filename) is not True:
    # generate bed file with expanded range 
    expand_range(f'../data/cagi5/{regulator}_combined_cagi.bed', output_filename, window = seqlen)

### Get sequences and their coordinates

In [94]:
fa_filename = f'../data/cagi5/{regulator}_combined_cagi_{seqlen}bp.fa'
coords_list, seqs_list = convert_bed_to_seq(output_filename, fa_filename, genomefile='/shared/jessica/ref/hg19/hg19.fa')

### Generate one hot seqs (WT & Mutant)

In [95]:
bad_lines = []
nonneg_df = pd.read_csv(output_filename, sep='\t', header=None) # load bed file with ref/alt alleles 
onehot_ref = []
onehot_alt = []
identifiers = []
pos_dict = {'+': int(seqlen/2-1), '-':int(seqlen/2)} # ix of allele pos in seq
for i,(chr_s_e, seq) in enumerate(zip(coords_list, seqs_list)):
    alt = ''
    strand = chr_s_e.split('(')[-1].split(')')[0]
    pos = pos_dict[strand] # get allele pos 
    if seq[pos] != nonneg_df.iloc[i, 3]:
        # check that nucleotide at pos is ref allele 
        print('Error in line ' + str(i))
        bad_lines.append(i)
    else:
        # get alt allele 
        alt = nonneg_df.iloc[i,4]
        onehot = dna_one_hot(seq) # one hot wt seq 
        mutated_onehot = onehot.copy() 
        mutated_onehot[pos] = dna_one_hot(alt)[0] # one hot mutant seq 
        onehot_ref.append(onehot)
        onehot_alt.append(mutated_onehot) 
        identifiers.append(nonneg_df.iloc[i,-1])

onehot_alt = np.array(onehot_alt)
onehot_ref = np.array(onehot_ref)

Error in line 0
Error in line 1
Error in line 2


In [96]:
# remove badlines 
included_df = nonneg_df[~nonneg_df.index.isin(bad_lines)]
included_df.to_csv(f'../data/cagi5/{regulator}_combined_cagi_{seqlen}bp_filt.bed', header=None, index=None, sep='\t')

### Write onehot seqs and identifiers to .h5 file


In [97]:
onehot_seqs_fh = f'../data/cagi5/{regulator}_onehot_{seqlen}bp.h5'
with h5py.File(onehot_seqs_fh, 'w') as fh:
    fh.create_dataset('ref', data=onehot_ref)
    fh.create_dataset('alt', data=onehot_alt)
    fh.create_dataset('identifier', data=identifiers)

## _LDLR_

### Generate BED file w/ expanded ranges

In [102]:
regulator = 'LDLR'

# output filename for expanded bed file
output_filename = f'../data/cagi5/{regulator}_combined_cagi_{seqlen}bp.bed'

if os.path.isfile(output_filename) is not True:
    # generate bed file with expanded range 
    expand_range(f'../data/cagi5/{regulator}_combined_cagi.bed', output_filename, window = seqlen)

### Get sequences and their coordinates

In [103]:
fa_filename = f'../data/cagi5/{regulator}_combined_cagi_{seqlen}bp.fa'
coords_list, seqs_list = convert_bed_to_seq(output_filename, fa_filename, genomefile='/shared/jessica/ref/hg19/hg19.fa')

### Generate one hot seqs (WT & Mutant)

In [104]:
bad_lines = []
nonneg_df = pd.read_csv(output_filename, sep='\t', header=None) # load bed file with ref/alt alleles 
onehot_ref = []
onehot_alt = []
identifiers = []
pos_dict = {'+': int(seqlen/2-1), '-':int(seqlen/2)} # ix of allele pos in seq
for i,(chr_s_e, seq) in enumerate(zip(coords_list, seqs_list)):
    alt = ''
    strand = chr_s_e.split('(')[-1].split(')')[0]
    pos = pos_dict[strand] # get allele pos 
    if seq[pos] != nonneg_df.iloc[i, 3]:
        # check that nucleotide at pos is ref allele 
        print('Error in line ' + str(i))
        bad_lines.append(i)
    else:
        # get alt allele 
        alt = nonneg_df.iloc[i,4]
        onehot = dna_one_hot(seq) # one hot wt seq 
        mutated_onehot = onehot.copy() 
        mutated_onehot[pos] = dna_one_hot(alt)[0] # one hot mutant seq 
        onehot_ref.append(onehot)
        onehot_alt.append(mutated_onehot) 
        identifiers.append(nonneg_df.iloc[i,-1])

onehot_alt = np.array(onehot_alt)
onehot_ref = np.array(onehot_ref)

In [105]:
# remove badlines 
included_df = nonneg_df[~nonneg_df.index.isin(bad_lines)]
included_df.to_csv(f'../data/cagi5/{regulator}_combined_cagi_{seqlen}bp_filt.bed', header=None, index=None, sep='\t')

### Write onehot seqs and identifiers to .h5 file


In [106]:
onehot_seqs_fh = f'../data/cagi5/{regulator}_onehot_{seqlen}bp.h5'
with h5py.File(onehot_seqs_fh, 'w') as fh:
    fh.create_dataset('ref', data=onehot_ref)
    fh.create_dataset('alt', data=onehot_alt)
    fh.create_dataset('identifier', data=identifiers)

## _SORT1_

### Generate BED file w/ expanded ranges

In [122]:
regulator = 'SORT1'

# output filename for expanded bed file
output_filename = f'../data/cagi5/{regulator}_combined_cagi_{seqlen}bp.bed'

if os.path.isfile(output_filename) is not True:
    # generate bed file with expanded range 
    expand_range(f'../data/cagi5/{regulator}_combined_cagi.bed', output_filename, window = seqlen)

### Get sequences and their coordinates

In [123]:
fa_filename = f'../data/cagi5/{regulator}_combined_cagi_{seqlen}bp.fa'
coords_list, seqs_list = convert_bed_to_seq(output_filename, fa_filename, genomefile='/shared/jessica/ref/hg19/hg19.fa')

### Generate one hot seqs (WT & Mutant)

In [126]:
bad_lines = []
nonneg_df = pd.read_csv(output_filename, sep='\t', header=None) # load bed file with ref/alt alleles 
onehot_ref = []
onehot_alt = []
identifiers = []
pos_dict = {'+': int(seqlen/2-1), '-':int(seqlen/2)} # ix of allele pos in seq
for i,(chr_s_e, seq) in enumerate(zip(coords_list, seqs_list)):
    alt = ''
    strand = chr_s_e.split('(')[-1].split(')')[0]
    pos = pos_dict[strand] # get allele pos 
    if seq[pos] != nonneg_df.iloc[i, 3]:
        # check that nucleotide at pos is ref allele 
        print('Error in line ' + str(i))
        bad_lines.append(i)
    else:
        # get alt allele 
        alt = nonneg_df.iloc[i,4]
        onehot = dna_one_hot(seq) # one hot wt seq 
        mutated_onehot = onehot.copy() 
        mutated_onehot[pos] = dna_one_hot(alt)[0] # one hot mutant seq 
        onehot_ref.append(onehot)
        onehot_alt.append(mutated_onehot) 
        identifiers.append(nonneg_df.iloc[i,-1])

onehot_alt = np.array(onehot_alt)
onehot_ref = np.array(onehot_ref)

Error in line 0
Error in line 1


In [127]:
# remove badlines 
included_df = nonneg_df[~nonneg_df.index.isin(bad_lines)]
included_df.to_csv(f'../data/cagi5/{regulator}_combined_cagi_{seqlen}bp_filt.bed', header=None, index=None, sep='\t')

### Write onehot seqs and identifiers to .h5 file


In [128]:
onehot_seqs_fh = f'../data/cagi5/{regulator}_onehot_{seqlen}bp.h5'
with h5py.File(onehot_seqs_fh, 'w') as fh:
    fh.create_dataset('ref', data=onehot_ref)
    fh.create_dataset('alt', data=onehot_alt)
    fh.create_dataset('identifier', data=identifiers)